# Data Preparation


This notebook explores the dataset:

Regulation of Brain Cognitive States through Auditory, Gustatory, and Olfactory Stimulation with Wearable Monitoring (v1.0.0)
from PhysioNet. https://physionet.org/content/brain-wearable-monitoring/1.0.0/

### Data Description
This dataset encompasses the outcomes of two distinct experiments, with each containing participants' correct/incorrect responses, accompanied by their response times, documented as 'n_back_responses'. Alongside these behavioral responses, the dataset incorporates physiological measurements collected from two Empatica devices, each affixed to either hand, as well as data obtained from a muse headband. 

To present the data in a structured manner, each participant's data is contained within a designated folder, numbered sequentially from A1 to A10 in Experiment 1, and B1 to B10 in Experiment 2. The dataset also includes the data from participants who were excluded for further analysis in [1]. Their folder names are named 'Excluded_ID_number' [1]. Their ID numbers are according to those presented in supplementary materials in [1]. The column “Start_time_unix” shows the initial time of the recording expressed as UNIX timestamp in UTC.  

The experiment settings and behavioral signals can be found in the ‘n_back_responses.csv’ file. The .csv file is in the following format:

The “ExperimentName” specifies the experiment of the study.
The “Instruction” specifies the presented n-back task at each trial.
The “Running [Block]” displays the applied actuator within each session.
In the designed n-back experiments, subjects were shown trials of stimulus (500 ms) along with a plus sign (fixation cross) for their response (1500 ms). The behavioral signals including the reaction time (RT) and correct/incorrect response are stored under the “Stimulus” and “Fixation” categories.

In experiment 1, the reaction times (milliseconds) associated with session 1 to session 4 are stored within “Fixation101.RT” to “Fixation104.RT” and “Stimulus101.RT” to “Stimulus104.RT”. The binary correct/incorrect responses associated with session 1 to session 4 are stored within “Fixation101.ACC” to “Fixation104.ACC” and “Stimulus101.ACC” to “Stimulus104.ACC”.

In experiment 2, the reaction times (milliseconds) associated with session 1 to session 3 are stored within “Fixation101.RT” to “Fixation103.RT” and “Stimulus101.RT” to “Stimulus103.RT”. The binary correct/incorrect responses associated with session 1 to session 3 are stored within “Fixation101.ACC” to “Fixation103.ACC” and “Stimulus101.ACC” to “Stimulus103.ACC”.

The EEG signals recorded from the Muse headband are also included in the dataset as 'EEG_recording.csv'. They include EEG channels from four locations: TP9, AF7, AF8, and TP10. The 2016 model Muse (MU-02) can also output RAW EEG data at 256Hz as well as being able to output RAW EEG from the right ear USB Auxiliary connector (named Right AUX in the csv files). The column (timestamps) shows the data points in UNIX format. EEG measurements are labeled as TP9, AF7, AF8, TP10, Right AUX.

The Empatica data encompasses a range of physiological aspects, including participants' electrodermal activity (EDA), heart rate (HR), blood volume pulses (BVP), skin surface temperature, Photoplethysmography (PPG), and 3-axis accelerometer data. These data types are stored as separate CSV files: 'EDA.csv', 'TEMP.csv', 'ACC.csv', 'BVP.csv', 'HR.csv', 'IBI.csv', and 'tags.csv'. Data recorded by Empatica wristbands from Left and Right hands are each represented by their own distinct CSV files, encompassing the following attributes:

- 'Left_EDA.csv' and 'Right_EDA.csv'
- 'Left_TEMP.csv' and 'Right_TEMP.csv'
- 'Left_ACC.csv' and 'Right_ACC.csv'
- 'Left_BVP.csv' and 'Right_BVP.csv'
- 'Left_HR.csv' and 'Right_HR.csv'
- 'Left_IBI.csv' and 'Right_IBI.csv'
- 'Left_tags.csv' and 'Right_tags.csv'

For clarity, a concise summary of the contents of the different data files is provided below:

- EDA: Measurements from the electrodermal activity (EDA) sensor expressed as micro siemens (μS). Values in the first column (EDA) show the EDA values. The column (start_time_unix) shows the initial time of the recording expressed as UNIX timestamp in UTC. The column (sampling_rate) shows the sample rate expressed in Hz.
- TEMP: Skin temperature (TEMP) measured from temperature sensor expressed degrees on the Celsius (°C) scale. Values in the first column (TEMP) show the TEMP values. The column (start_time_unix) shows the initial time of the recording expressed as UNIX timestamp in UTC. The column (sampling_rate) shows the sample rate expressed in Hz.
- ACC: Data from 3-axis accelerometer sensor. The accelerometer is configured to measure acceleration in the range [-2g, 2g]. Therefore, the unit in this file is 1/64g. Data from x, y, and z axis are labeled respectively (i.e., ACC_X, ACC_Y, and ACC_Z).  The column (start_time_unix) shows the initial time of the recording expressed as UNIX timestamp in UTC. The column (sampling_rate) shows the sample rate expressed in Hz.
- BVP: Blood volume pulses (BVP) measured from a photoplethysmograph. Values in the first column (BVP) show the BVP values. The column (start_time_unix) shows the initial time of the recording expressed as UNIX timestamp in UTC. The column (sampling_rate) shows the sample rate expressed in Hz.
- HR: Average heart rate (HR) extracted from the BVP signal. Values in the first column (HR) show the HR values. The column (start_time_unix) shows the initial time of the recording expressed as UNIX timestamp in UTC. The column (sampling_rate) shows the sample rate expressed in Hz.
- IBI: Time between individuals heart beats extracted from the BVP signal. The first column (IBI_time) shows the time (with respect to the initial time) of the detected inter-beat interval expressed in seconds (s). The second column (IBI_intervals) shows the duration in seconds (s) of the detected inter-beat interval (i.e., the distance in seconds from the previous beat). No sample rate is needed for this file.
- tags: Event mark times in each Empatica device. Each row corresponds to a physical button press on the device; at the same time as the status LED is first illuminated. The time is expressed as a UNIX timestamp in UTC, and it is synchronized with the initial time of the recordings indicated in the related data files from the corresponding session.
Due to the inadvertent receipt of certain tags during experiments, the correct and unified tags are also included as 'tags.csv'.

[1] Fekri Azgomi, Hamid, et al. "Regulation of brain cognitive states through auditory, gustatory, and olfactory stimulation with wearable monitoring." Scientific Reports 13.1 (2023): 12399. https://www.nature.com/articles/s41598-023-37829-z 

In [1]:
import os
from pathlib import Path
import re
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.signal import welch, find_peaks
from scipy.stats import iqr
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Assumptions (as specified in data description):
# - Each participant folder (A1..A10, B1..B10, Excluded_...) contains:
#     n_back_responses.csv
#     EEG_recordings.csv
#     Left_ACC.csv, Right_ACC.csv, Left_BVP.csv, Right_BVP.csv, Left_EDA.csv, Right_EDA.csv, Left_HR.csv, Right_HR.csv,
#     Left_IBI.csv, Right_IBI.csv, Left_TEMP.csv, Right_TEMP.csv, tags.csv (or Left_tags/Right_tags)
# - Empatica CSVs have columns (ACC_X,ACC_Y,ACC_Z,start_time_unix,sampling_rate, etc.)
# - Muse EEG CSV has 'timestamps' (UNIX seconds or ms) and channels TP9, AF7, AF8, TP10, Right AUX
# - Trial structure: trials spaced every 2.0 s per session block (500 ms stimulus + 1500 ms fixation)
# - Reaction times in Stimulus###.RT / Fixation###.RT are in milliseconds (ms) and ACC columns indicate correctness (1==correct)

Data stored locally

In [3]:
# ---------- dataset folder ----------
BASE_PATH = Path(r"C:\Users\Kavisha\OneDrive - Johns Hopkins\EN.585.771_BME_DATASCIENCE\Capstone_Project\App\BME_DATA_SCIENCE_CAPSTONE_PROJECT\Data\Brain_Cognitive_States_Data")
EXP1 = BASE_PATH / "Experiment_1"
EXP2 = BASE_PATH / "Experiment_2"
OUT_DIR = BASE_PATH / "processed_features_v2"
OUT_DIR.mkdir(exist_ok=True, parents=True)

Functions for handling files 

In [ ]:
def load_eeg(eeg_path):
    df = pd.read_csv(eeg_path)
    tcol = next((c for c in df.columns if 'time' in c.lower() or 'timestamp' in c.lower()), None)
    if tcol is None:
        raise ValueError("EEG file missing timestamps")
    
    if df[tcol].median() > 1e9:  # ms in UNIX ms
        df['time_s'] = df[tcol] / 1000.0
    else:
        df['time_s'] = df[tcol]
    df = df.set_index('time_s').sort_index()
    return df

In [ ]:
def load_empatica_file(path, kind):
    
    df = pd.read_csv(path)
    if 'start_time_unix' not in [c.lower() for c in df.columns]:
        raise ValueError(f"{path.name} missing start_time_unix column")
    # ensure column names as original (case-sensitive) accessible
    # find the actual column names for start_time_unix and sampling_rate if they have different case
    start_col = next(c for c in df.columns if c.lower() == 'start_time_unix')
    sr_col = next((c for c in df.columns if c.lower() == 'sampling_rate'), None)
    st = float(df[start_col].dropna().iloc[0])
    sr = float(df[sr_col].dropna().iloc[0]) if sr_col else None

    if kind.lower() == 'acc':
        # expect ACC_X,ACC_Y,ACC_Z present
        # create time axis from index and sampling rate
        if sr is None:
            raise ValueError(f"ACC file {path} missing sampling_rate")
        n = len(df)
        times = st + (np.arange(n) / sr)
        out = df.copy().reset_index(drop=True)
        out['time_s'] = times
        out = out.set_index('time_s').sort_index()
        # keep ACC_X/Y/Z
        cols = [c for c in out.columns if c.lower() in ('acc_x','acc_y','acc_z')]
        return out[cols].astype(float)
    elif kind.lower() in ('bvp','eda','hr','temp'):
        if sr is None:
            raise ValueError(f"{path} missing sampling_rate")
        n = len(df)
        times = st + (np.arange(n) / sr)
        # pick the measurement column (first column that's not start_time_unix or sampling_rate)
        meas_col = next((c for c in df.columns if c.lower() not in ('start_time_unix','sampling_rate')), df.columns[0])
        out = pd.DataFrame({meas_col: df[meas_col].values}, index=times)
        out.index.name = 'time_s'
        return out.astype(float)
    elif kind.lower() == 'ibi':
        # expects IBI_time (s) and IBI_intervals (s)
        # absolute times = start_time_unix + IBI_time
        ibi_time_col = next((c for c in df.columns if c.lower() == 'ibi_time'), None)
        ibi_int_col = next((c for c in df.columns if 'ibi_intervals' in c.lower() or 'ibi_interval' in c.lower()), None)
        if ibi_time_col is None:
            # try lowercase names
            raise ValueError(f"IBI file {path} missing IBI_time column")
        abs_times = st + df[ibi_time_col].astype(float).values
        out = pd.DataFrame({'IBI_interval': df[ibi_int_col].astype(float).values}, index=abs_times)
        out.index.name = 'time_s'
        return out
    else:
        raise ValueError("Unknown empatica kind")

In [ ]:
# ---------- Behavior parsing (uses the actual Stimulus###.RT & Fixation###.RT style columns) ----------
def load_behavior(participant_folder):
    bf = participant_folder / 'n_back_responses.csv'
    if not bf.exists():
        raise FileNotFoundError(f"Behavior file missing for {participant_folder}")
    df = pd.read_csv(bf)
    return df

In [ ]:
def get_session_tags(participant_folder):
    # prefer unified tags.csv, else Left_tags or Right_tags, else None
    for fname in ['tags.csv','Tags.csv','left_tags.csv','Left_tags.csv','right_tags.csv','Right_tags.csv']:
        p = participant_folder / fname
        if p.exists():
            arr = pd.read_csv(p, header=None).iloc[:,0].astype(float).tolist()
            return arr
    return []

In [ ]:
def parse_behavior_events(behavior_df, tags_list, experiment_label):
    """
    Convert stimulus/fixation RT & ACC columns into event rows with absolute timestamps (seconds).
    - behavior_df: per-participant n_back_responses.csv
    - tags_list: list of session start UNIX times in seconds (tags.csv)
    - experiment_label: 'Experiment_1' or 'Experiment_2'
    
    Returns DataFrame with columns:
    event_time_s, trial_type, session_idx, trial_idx, RT_s, ACC
    """
    # ---------------------------
    # 1. Identify session type column
    # ---------------------------
    if "Running [Block]" in behavior_df.columns:
        block_col = "Running [Block]"
    elif "Running[Block]" in behavior_df.columns:
        block_col = "Running[Block]"
    else:
        block_col = None

    if behavior_df is None or behavior_df.shape[0] == 0:
        return pd.DataFrame(columns=[
            'event_time_s', 'trial_type', 'session_idx', 'trial_idx',
            'RT_s', 'ACC', 'session_type'
        ])
        
    # ---------------------------
    # 2. RT and ACC column detection
    # ---------------------------
    stim_rt_cols = sorted([c for c in behavior_df.columns 
                           if re.match(r'(?i)stimulus[0-9]{3}\.RT$', c) or re.match(r'(?i)stimulus[0-9]{3}_RT$', c)])

    stim_acc_cols = sorted([c for c in behavior_df.columns 
                            if re.match(r'(?i)stimulus[0-9]{3}\.ACC$', c) or re.match(r'(?i)stimulus[0-9]{3}_ACC$', c)])

    fix_rt_cols = sorted([c for c in behavior_df.columns 
                          if re.match(r'(?i)fixation[0-9]{3}\.RT$', c) or re.match(r'(?i)fixation[0-9]{3}_RT$', c)])

    fix_acc_cols = sorted([c for c in behavior_df.columns 
                           if re.match(r'(?i)fixation[0-9]{3}\.ACC$', c) or re.match(r'(?i)fixation[0-9]{3}_ACC$', c)])
       
    # ---------------------------
    # 3. Build maps for each session code
    # ---------------------------
    stim_map = {}
    for c in stim_rt_cols:
        code = int(re.search(r'([0-9]{3})', c).group(1))
        stim_map.setdefault(code, {})['rt'] = behavior_df[c].values

    for c in stim_acc_cols:
        code = int(re.search(r'([0-9]{3})', c).group(1))
        stim_map.setdefault(code, {})['acc'] = behavior_df[c].values

    fix_map = {}
    for c in fix_rt_cols:
        code = int(re.search(r'([0-9]{3})', c).group(1))
        fix_map.setdefault(code, {})['rt'] = behavior_df[c].values

    for c in fix_acc_cols:
        code = int(re.search(r'([0-9]{3})', c).group(1))
        fix_map.setdefault(code, {})['acc'] = behavior_df[c].values
        
    # ---------------------------
    # 4. Expected sessions (Exp1 = 4, Exp2 = 3)
    # ---------------------------
    if '1' in str(experiment_label):
        expected_codes = [101, 102, 103, 104]
    else:
        expected_codes = [101, 102, 103]
        
    # ---------------------------
    # 5. Session start timestamps
    # ---------------------------
    if len(tags_list) == 0:
        st_col = next((c for c in behavior_df.columns if c.lower() == 'start_time_unix'), None)
        if st_col:
            base_time = float(behavior_df[st_col].dropna().iloc[0])
            tags_list = [base_time + i*1.0 for i in range(len(expected_codes))]
        else:
            tags_list = [0.0] * len(expected_codes)
            
    # ---------------------------
    # 6. Build event rows
    # ---------------------------
    events = []
    for sidx, code in enumerate(expected_codes):

        session_start = tags_list[sidx] if sidx < len(tags_list) else tags_list[-1]

        stim_entry = stim_map.get(code, {})
        fix_entry = fix_map.get(code, {})

        n_trials = len(stim_entry.get('rt', [])) if 'rt' in stim_entry \
                   else len(fix_entry.get('rt', [])) if 'rt' in fix_entry else 0

        for j in range(n_trials):

            # Extract session type (same row for stimulus & fixation)
            session_type_val = None
            if block_col is not None and j < len(behavior_df):
                session_type_val = behavior_df[block_col].iloc[j]

            # ------------------ Stimulus ------------------
            stim_onset = session_start + j * 2.0

            # RT
            rt_s = None
            if 'rt' in stim_entry and j < len(stim_entry['rt']):
                try:
                    rt_ms = float(stim_entry['rt'][j])
                    rt_s = rt_ms / 1000.0 if (rt_ms > 0 and np.isfinite(rt_ms)) else None
                except:
                    rt_s = None

            # ACC
            acc_s = None
            if 'acc' in stim_entry and j < len(stim_entry['acc']):
                try:
                    acc_s = int(stim_entry['acc'][j])
                except:
                    acc_s = None

            events.append({
                'event_time_s': stim_onset + rt_s if rt_s else stim_onset,
                'trial_type': 'stimulus',
                'session_idx': sidx+1,
                'trial_idx': j+1,
                'RT_s': rt_s if rt_s else np.nan,
                'ACC': acc_s,
                'session_type': session_type_val,
            })

            # ------------------ Fixation ------------------
            fix_onset = stim_onset + 0.5

            rt_f = None
            if 'rt' in fix_entry and j < len(fix_entry['rt']):
                try:
                    rt_ms = float(fix_entry['rt'][j])
                    rt_f = rt_ms / 1000.0 if (rt_ms > 0 and np.isfinite(rt_ms)) else None
                except:
                    rt_f = None

            acc_f = None
            if 'acc' in fix_entry and j < len(fix_entry['acc']):
                try:
                    acc_f = int(fix_entry['acc'][j])
                except:
                    acc_f = None

            events.append({
                'event_time_s': fix_onset + rt_f if rt_f else fix_onset,
                'trial_type': 'fixation',
                'session_idx': sidx+1,
                'trial_idx': j+1,
                'RT_s': rt_f if rt_f else np.nan,
                'ACC': acc_f,
                'session_type': session_type_val,
            })
            
    # ---------------------------
    # 7. Final dataframe
    # ---------------------------
    evdf = pd.DataFrame(events)
    evdf = evdf.sort_values('event_time_s').reset_index(drop=True)

    return evdf    

In [ ]:
# ---------- Feature extraction ----------
def bandpower(arr, fs, band):
    if len(arr) < 4 or np.all(np.isnan(arr)):
        return np.nan
    f,Pxx = welch(arr, fs=fs, nperseg=min(256, len(arr)))
    mask = (f>=band[0]) & (f<=band[1])
    if mask.sum()==0:
        return 0.0
    return np.trapz(Pxx[mask], f[mask])

BANDS = {'delta':(1,4),'theta':(4,8),'alpha':(8,13),'beta':(13,30),'gamma':(30,45)}

In [ ]:
def acc_magnitude(acc_df):
    arr = acc_df[['ACC_X','ACC_Y','ACC_Z']].astype(float).values
    return np.linalg.norm(arr, axis=1)

In [ ]:
def ibi_features(ibi_df):
    arr = ibi_df['IBI_interval'].dropna().values
    if len(arr)==0:
        return {'ibi_count':0,'ibi_sdnn':np.nan,'ibi_rmssd':np.nan}
    sdnn = np.std(arr, ddof=1)
    rmssd = np.sqrt(np.mean(np.diff(arr)**2)) if len(arr)>1 else np.nan
    return {'ibi_count':len(arr),'ibi_sdnn':sdnn,'ibi_rmssd':rmssd}

In [ ]:
def extract_window_features(common_time, resampled, fs=10., win=5.0, step=2.5):
    rows=[]
    starts = np.arange(common_time[0], common_time[-1]-win+1e-6, step)
    for s in starts:
        e = s+win
        row = {'window_start':s,'window_end':e,'window_center':s+win/2}
        # EEG bandpower per channel 
        if 'eeg' in resampled:
            seg = resampled['eeg'].loc[(resampled['eeg'].index>=s)&(resampled['eeg'].index<e)]
            if seg.shape[0]>2:
                for ch in seg.columns:
                    for bname, br in BANDS.items():
                        row[f"{ch}_{bname}"] = bandpower(seg[ch].values, fs=fs, band=br)
            else:
                for ch in ['TP9','AF7','AF8','TP10','Right AUX']:
                    for bname in BANDS:
                        row[f"{ch}_{bname}"] = np.nan
        # EDA
        for side in ['left_eda','right_eda']:
            if side in resampled:
                seg = resampled[side].loc[(resampled[side].index>=s)&(resampled[side].index<e)]
                if seg.shape[0]>0:
                    col = seg.columns[0]
                    vals = seg[col].astype(float).values
                    row[f"{side}_mean"] = np.nanmean(vals)
                    row[f"{side}_std"] = np.nanstd(vals)
                    try:
                        pks,_ = find_peaks(vals, height=np.nanmean(vals)+0.5*np.nanstd(vals))
                        row[f"{side}_n_peaks"] = len(pks)
                    except Exception:
                        row[f"{side}_n_peaks"] = 0
                else:
                    row[f"{side}_mean"]=np.nan; row[f"{side}_std"]=np.nan; row[f"{side}_n_peaks"]=0
        # BVP -> HR approx
        for side in ['left_bvp','right_bvp']:
            if side in resampled:
                seg = resampled[side].loc[(resampled[side].index>=s)&(resampled[side].index<e)]
                if seg.shape[0]>3:
                    vals = seg.iloc[:,0].astype(float).values
                    try:
                        pks,_ = find_peaks(vals, distance=int(fs*0.4))
                        if len(pks)>=2:
                            idxs = seg.index.values
                            mean_dt = np.nanmean(np.diff(idxs)) if len(idxs)>1 else 1.0/fs
                            rr = np.diff(pks) * mean_dt
                            hr = 60.0/rr
                            row[f"{side}_hr_mean"] = np.nanmean(hr)
                            row[f"{side}_hr_std"] = np.nanstd(hr)
                            row[f"{side}_n_beats"] = len(pks)
                        else:
                            row[f"{side}_hr_mean"]=np.nan; row[f"{side}_hr_std"]=np.nan; row[f"{side}_n_beats"]=len(pks)
                    except Exception:
                        row[f"{side}_hr_mean"]=np.nan; row[f"{side}_hr_std"]=np.nan; row[f"{side}_n_beats"]=0
                else:
                    row[f"{side}_hr_mean"]=np.nan; row[f"{side}_hr_std"]=np.nan; row[f"{side}_n_beats"]=0
        # ACC metrics
        for side in ['left_acc','right_acc']:
            if side in resampled:
                seg = resampled[side].loc[(resampled[side].index>=s)&(resampled[side].index<e)]
                if seg.shape[0]>0:
                    mag = acc_magnitude(seg)
                    row[f"{side}_mag_mean"] = np.nanmean(mag)
                    row[f"{side}_mag_std"] = np.nanstd(mag)
                    row[f"{side}_mag_iqr"] = iqr(mag) if len(mag)>0 else np.nan
                else:
                    row[f"{side}_mag_mean"]=np.nan; row[f"{side}_mag_std"]=np.nan; row[f"{side}_mag_iqr"]=np.nan
        # TEMP
        for side in ['left_temp','right_temp']:
            if side in resampled:
                seg = resampled[side].loc[(resampled[side].index>=s)&(resampled[side].index<e)]
                if seg.shape[0]>0:
                    col = seg.columns[0]
                    row[f"{side}_mean"] = np.nanmean(seg[col].astype(float).values)
                    row[f"{side}_std"] = np.nanstd(seg[col].astype(float).values)
                else:
                    row[f"{side}_mean"]=np.nan; row[f"{side}_std"]=np.nan
        # IBI HRV
        for side in ['left_ibi','right_ibi']:
            if side in resampled:
                ibi_seg = resampled[side].loc[(resampled[side].index>=s)&(resampled[side].index<e)]
                if ibi_seg.shape[0]>0:
                    h = ibi_features(ibi_seg)
                    row.update({f"{side}_{k}":v for k,v in h.items()})
                else:
                    row[f"{side}_ibi_count"]=0; row[f"{side}_ibi_sdnn"]=np.nan; row[f"{side}_ibi_rmssd"]=np.nan
        rows.append(row)
    return pd.DataFrame(rows)

In [ ]:
# ---------- Per-participant processing ----------
def process_participant(participant_folder, common_fs=10.0, win_s=5.0, step_s=2.5):
    print("Processing:", participant_folder.name)

    # ---------------------------
    # 1. Load behavior and tags
    # ---------------------------
    behavior_df = load_behavior(participant_folder)
    tags = get_session_tags(participant_folder)

    # ---------------------------
    # 2. Parse events (includes session_type from your updated function)
    # ---------------------------
    events = parse_behavior_events(
        behavior_df, tags, participant_folder.parent.name
    )

    # ---------------------------
    # 3. Load signals (EEG + Empatica)
    # ---------------------------
    signals = {}

    # EEG
    eeg_p = participant_folder / 'EEG_recordings.csv'
    if eeg_p.exists():
        signals['eeg'] = load_eeg(eeg_p)

    # Empatica left and right wrist data
    for side in ['Left', 'Right']:
        for kind in ['ACC', 'BVP', 'EDA', 'HR', 'IBI', 'TEMP']:
            p = participant_folder / f"{side}_{kind}.csv"
            if p.exists():
                try:
                    signals[f"{side.lower()}_{kind.lower()}"] = load_empatica_file(
                        p, kind.lower()
                    )
                except Exception as e:
                    print(f"Warning loading {p.name}: {e}")

    # ---------------------------
    # 4. Build common time axis
    # ---------------------------
    t_mins, t_maxs = [], []
    for df in signals.values():
        t_mins.append(df.index.min())
        t_maxs.append(df.index.max())

    if len(t_mins) == 0:
        raise ValueError(f"No signals found for {participant_folder.name}")

    overall_min = min([t for t in t_mins if pd.notna(t)])
    overall_max = max([t for t in t_maxs if pd.notna(t)])

    common_time = np.arange(overall_min, overall_max, 1.0 / common_fs)

    # ---------------------------
    # 5. Resample all signals to common_time
    # ---------------------------
    resampled = {}
    for k, df in signals.items():
        try:
            df2 = df.copy()
            df2.index = df2.index.astype(float)
            resampled[k] = df2.reindex(
                common_time, method='nearest', tolerance=1.0 / common_fs
            )
        except Exception:
            # fallback interpolation if needed
            resampled[k] = df.reindex(common_time).interpolate().ffill().bfill()

    # ---------------------------
    # 6. Extract window-level features
    # ---------------------------
    feats = extract_window_features(
        common_time, resampled, fs=common_fs, win=win_s, step=step_s
    )
    feats['participant'] = participant_folder.name

    # Prepare behavioral summary per-window
    feats['n_events_in_window'] = 0
    feats['mean_RT'] = np.nan
    feats['prop_correct'] = np.nan
    feats['session_type'] = None  # <-- NEW COLUMN

    # ---------------------------
    # 7. Add behavior + session_type to windows
    # ---------------------------
    for idx, win in feats.iterrows():
        s = win['window_start']
        e = win['window_end']

        evs = events[(events['event_time_s'] >= s) & (events['event_time_s'] < e)]
        feats.at[idx, 'n_events_in_window'] = len(evs)

        # Mean RT
        if len(evs) > 0:
            rts = evs['RT_s'].dropna().astype(float).values
            feats.at[idx, 'mean_RT'] = np.nanmean(rts) if len(rts) > 0 else np.nan

            accs = evs['ACC'].dropna().astype(float).values
            feats.at[idx, 'prop_correct'] = (
                np.nanmean((accs == 1).astype(float)) if len(accs) > 0 else np.nan
            )

            # ---- Assign session type for the window ----
            if 'session_type' in evs.columns:
                # pick the most common session type in that window
                st_series = evs['session_type'].dropna()
                if len(st_series) > 0:
                    feats.at[idx, 'session_type'] = st_series.mode().iloc[0]
                else:
                    feats.at[idx, 'session_type'] = None
            else:
                feats.at[idx, 'session_type'] = None

    return {
        'participant': participant_folder.name,
        'features': feats,
        'events': events,
        'resampled': resampled
    }


In [ ]:
#  ---------- Process all participants in an experiment ----------
def process_experiment(exp_folder, out_dir=OUT_DIR, participants=None):
    results = {}
    all_feats=[]
    
    for sub in sorted(exp_folder.iterdir()):
        if not sub.is_dir(): continue
        if participants and sub.name not in participants: continue
        try:
            res = process_participant(sub)
            results[sub.name] = res
            all_feats.append(res['features'])
            # save per-subject
            res['features'].to_parquet(Path(out_dir)/f"{sub.name}_features.parquet")
            res['events'].to_csv(Path(out_dir)/f"{sub.name}_events.csv", index=False)
            print(f"Saved {sub.name} features/events")
        except Exception as e:
            print(f"Failed {sub.name}: {e}")
    if len(all_feats)==0:
        return pd.DataFrame(), results
    combined = pd.concat(all_feats, ignore_index=True)
    combined.to_parquet(Path(out_dir)/f"{exp_folder.name}_all_features.parquet")
    combined.to_csv(Path(out_dir)/f"{exp_folder.name}_all_features.csv", index=False)
    return combined, results

In [ ]:
# ---------- Run for both experiments ----------
exp1_feats, exp1_res = process_experiment(EXP1)
exp2_feats, exp2_res = process_experiment(EXP2)
full_df = pd.concat([exp1_feats, exp2_feats], ignore_index=True)
full_df.to_parquet(OUT_DIR/"all_participants_features.parquet")
full_df.to_csv(OUT_DIR/"all_participants_features.csv", index=False)
print("All done. Saved combined features to:", OUT_DIR)